# Projet 10 : Développez un chatbot pour réserver des vacances

## Importation des librairie

In [1]:
import os
import uuid
import time
import json
import numpy as np 
import pandas as pd
from azure.cognitiveservices.language.luis.authoring import LUISAuthoringClient
from azure.cognitiveservices.language.luis.runtime import LUISRuntimeClient
from msrest.authentication import CognitiveServicesCredentials
from azure.cognitiveservices.language.luis.authoring.models import ApplicationCreateObject
import matplotlib.pyplot as plt 
import seaborn as sns 
import plotly.express as px
pd.options.mode.chained_assignment = None

## Dèfinir le nouveau jeu de données

Comme ce projet est itératif, nous avons limité les fonctionnalités de la V1 du chatbot. La V1 devra pouvoir identifier dans la demande de l’utilisateur les cinq éléments suivants :

>- Ville de départ
>- Ville de destination
>- Date aller souhaitée du vol
>- Date retour souhaitée du vol
>- Budget maximum pour le prix total des billets

Pour créer ma nouvelle base de données, j'aurai besoin des champs suivant :
>- text, 
>- intentName,
>- entityLabels,
>- startCharIndex,
>- endCharIndex,
>- entityName

## Définir mes fonctions

In [2]:
def entity_char(entity_name, frame):
    # Charger une partie de l'entité du cadre
    entities = frame['labels']['frames'][0]['info']
    str_entire = frame['labels']['frames'][0]['info'][entity_name][0]['val'].lower()
    return str_entire.split('.')[0]

def entity_data(entity_char, text, entity_name):
    start_index = max(text.find(entity_char), 0)
    end_char_index = max(text.find(entity_char) + len(entity_char) - 1, 0)
    return {'startCharIndex': start_index, 'endCharIndex': end_char_index, 'entityName': entity_name}

def entity_data_dict(frame, pf_entities, intent_name):
    text = frame['text'].lower()
    entities = frame['labels']['frames'][0]['info']    
    entity_list = []
    for entity_name in pf_entities:
        # Vérifiez si des entités existent
        if entity_name in entities.keys():
            # Vérifiez si la valeur n’est pas -1
            if entity_char(entity_name, frame)!='-1':
                entity_list.append(entity_data(entity_char(entity_name, frame), text, entity_name))
    dict_output = {
        "text": text,
        "intentName": intent_name,
        "entityLabels": entity_list
    }
    return dict_output

## Charger mon jeu de données brute

In [3]:
dataframe = pd.read_json("data/frames.json")
# Dèfinir les entités à extraire
pf_entities = ['or_city','dst_city','str_date','end_date','budget']

## Créer mon dictionnaire

In [4]:
dictionnaire = [
    entity_data_dict(
        dataframe['turns'][x][0], pf_entities, "BookFlightIntent"
    ) for x in range(0, len(dataframe['turns']))
]

## Créer mes nouveaux jeux de données

In [5]:
if not os.path.exists("data/frames_train.json"):
    frames_train = dictionnaire[:-40]

    with open('data/frames_train.json', 'w') as f:
        f.write(json.dumps(frames_train))
else:
    with open("data/frames_train.json") as f:
        frames_train = json.load(f)

if not os.path.exists("data/frames_test.json"):
    frames_test = dictionnaire[-40:]

    with open('data/frames_test.json', 'w') as f:
        f.write(json.dumps(frames_test))
else:
    with open("data/frames_test.json.json") as f:
        frames_test = json.load(f)

## Explication synthétique du modèle LUIS

Le langage de compréhension (LUIS) nous permet d’appliquer un traitement en langage naturel au texte en langage naturel des conversations d’un utilisateur afin d’en prédire le sens général, et d’en extraire des informations détaillées et pertinentes.

Une application LUIS stocke le modèle de traitement en langage naturel contenant les intentions, les entités et les exemples d'énoncés.

## Créer une application LUIS

In [6]:
# Créez des variables pour stocker votre clé de création et les noms de vos ressources
authoring_key = 'd06c7abdf7444166b6fd614a51cb285c'
authoring_endpoint = 'https://authojk.cognitiveservices.azure.com/'
prediction_key = '5f66a9bb70e14d7892bddee92bd9196c'
prediction_endpoint = 'https://predictjkrai.cognitiveservices.azure.com/'

# Créez des variables pour stocker vos points de terminaison, le nom de l'application, la version et le nom de l'intention
# J'ai utilisé un UUID pour éviter les collisions de noms
app_name = "FlyMe-LUIS" + str(uuid.uuid4())
version_id = "0.1"
intent_name = "BookFlightIntent"

In [7]:

# Créer un client
client = LUISAuthoringClient(authoring_endpoint, CognitiveServicesCredentials(authoring_key))

# Définir les bases de l’application
app_definition = ApplicationCreateObject(name=app_name, initial_version_id=version_id, culture='en-us')

# Créer une application
app_id = client.apps.add(app_definition)

# Obtenir l’identifiant de l’application - nécessaire pour tous les autres changements
print("Created LUIS app with ID {}".format(app_id))

Created LUIS app with ID d5aedfec-a268-4ada-89d3-b002561d9180


L’objectif principal du modèle d’application LUIS est l’intention. L’intention est l'objectif exprimé dans l'énoncé d'un utilisateur. Un utilisateur peut poser une question ou émettre un énoncé en souhaitant obtenir une réponse prévue.

In [8]:
# Ajouter une nouvelle intention
client.model.add_intent(app_id, version_id, intent_name)

'e78d4bd2-1af5-4bda-80af-a34fd7f2b312'

Les entités, elles, ne sont pas obligatoires, elles sont présentes dans la plupart des applications. L’entité extrait des informations à partir de l’énoncé utilisateur, qui sont nécessaires pour répondre à l’intention de l’utilisateur. Il existe plusieurs types d’entités prédéfinies et personnalisées, chacune avec leurs propres modèles DTO (Data Transformation Object). Les entités prédéfinies courantes à ajouter à notre application incluent number, datetimeV2, geographyV2 et ordinal.

Il est important de savoir que les entités ne sont pas marquées avec une intention. Elles peuvent s’appliquer à de nombreuses intentions. Seuls les exemples d’énoncés utilisateur sont marqués pour une intention unique spécifique.

In [9]:
# Ajouter les entités
model_id = client.model.add_entity(app_id, version_id, name="or_city")
model_id = client.model.add_entity(app_id, version_id, name="dst_city")
model_id = client.model.add_entity(app_id, version_id, name="end_date")
model_id = client.model.add_entity(app_id, version_id, name="str_date")
model_id = client.model.add_entity(app_id, version_id, name="budget")

Pour déterminer l’intention d’un énoncé et extraire des entités, l’application a besoin d’exemples d’énoncés. Les exemples doivent cibler une intention spécifique et unique, et doivent marquer toutes les entités personnalisées. Les entités prédéfinies n’ont pas besoin d’être marquées.

In [10]:
# Ajouter le jeu de données d'entraînement
for x in range(0, len(frames_train)):
    try:
        client.examples.add(app_id, version_id, frames_train[x])
    except Exception as e:
        pass

## Entraînement de l'application

Maintenant que le modèle est créé, l’application LUIS va être entraînée pour cette version du modèle. Un modèle entraîné peut être utilisé dans un conteneur ou publié dans les emplacements intermédiaires ou produits.

Un petit modèle sera entraîné très rapidement. Pour les applications de niveau production, l’entraînement de l’application va devoir inclure un appel d’interrogation à la méthode get_status pour déterminer si l’entraînement a réussi. Tous les objets doivent réussir pour que l’entraînement soit considéré comme terminé.

In [11]:
print("Je vais commencer à former notre application...")

async_training = client.train.train_version(app_id, version_id)
is_trained = async_training.status == "UpToDate"

trained_status = ["UpToDate", "Success"]
while not is_trained:
    time.sleep(1)
    status = client.train.get_status(app_id, version_id)
    is_trained = all(m.details.status in trained_status for m in status)
    
print("Entraînement réalisé")

Je vais commencer à former notre application...
Entraînement réalisé


Je vais publier l’application LUIS. Cela va me permettre de la publier à l’emplacement spécifié au point de terminaison. Notre application cliente utilise ce point de terminaison afin d’envoyer des énoncés utilisateur pour la prédiction de l’intention et l’extraction d’entité.

In [12]:
print("Je commence à publier notre application...")
client.apps.update_settings(app_id, is_public=True)
publish_result = client.apps.publish(app_id, version_id, is_staging=False)
publish_result.as_dict()
endpoint = publish_result.endpoint_url + "?subscription-key=" + authoring_key + "&q="
print("Notre application est publiée. Je peux maintenant aller la tester sur\n{}".format(endpoint))

Je commence à publier notre application...
Notre application est publiée. Je peux maintenant aller la tester sur
https://australiaeast.api.cognitive.microsoft.com/luis/v2.0/apps/d5aedfec-a268-4ada-89d3-b002561d9180?subscription-key=d06c7abdf7444166b6fd614a51cb285c&q=


Je vais créer une requête à adresser au runtime de prédiction. Cette requête va me retourner un objet PredictionResponse.

In [13]:
runtime_credentials = CognitiveServicesCredentials(prediction_key)
client_runtime = LUISRuntimeClient(endpoint=prediction_endpoint, credentials=runtime_credentials)

# Test avec un seul énoncé
query = "looking to go from Paris to marseille. book me for july 15 to 28. let me know if its more than 800 because thats all i can afford"
result = client_runtime.prediction.resolve(app_id, query, verbose=False)

print("Requête: {}".format(query))
print("Entités détectées:")
for entity in result.entities:
    print(
        "\t-> Entité '{}' (type: {}, score:{:d}%)".format(
            entity.entity,
            entity.type,
            int(entity.additional_properties['score']*100)
        ))

Requête: looking to go from Paris to marseille. book me for july 15 to 28. let me know if its more than 800 because thats all i can afford
Entités détectées:
	-> Entité 'marseille' (type: dst_city, score:99%)
	-> Entité '28' (type: end_date, score:99%)
	-> Entité 'paris' (type: or_city, score:99%)
	-> Entité 'july 15' (type: str_date, score:99%)


A prèsent, je vais calculé le score moyen de chaque entité dans notre fichier de test.

In [14]:
calc_budget = 0
score_budget = 0
calc_dst_city = 0
score_dst_city = 0
calc_or_city = 0
score_or_city = 0
calc_end_date = 0
score_end_date = 0
calc_str_date = 0
score_str_date = 0

for text in frames_test:
    query = text['text']
    result = client_runtime.prediction.resolve(app_id, query, verbose=False)
    for entity in result.entities:
        if "budget" == entity.type:
            calc_budget += 1
            score_budget += entity.additional_properties["score"] * 100
        if "dst_city" == entity.type:
            calc_dst_city += 1
            score_dst_city += entity.additional_properties["score"] * 100
        if "end_date" == entity.type:
            calc_end_date += 1
            score_end_date += entity.additional_properties["score"] * 100
        if "or_city" == entity.type:
            calc_or_city += 1
            score_or_city += entity.additional_properties["score"] * 100
        if "str_date" == entity.type:
            calc_str_date += 1
            score_str_date += entity.additional_properties["score"] * 100
print("Score budgétaire moyen: {}".format(score_budget / calc_budget))
print("Score ville de destination moyen: {}".format(score_dst_city / calc_dst_city))
print("Score date d'arrivè moyen: {}".format(score_end_date / calc_end_date))
print("Score ville d'origine moyen: {}".format(score_or_city / calc_or_city))
print("Score date de départ d'origine moyen: {}".format(score_str_date / calc_str_date))

Score budgétaire moyen: 90.669705
Score ville de destination moyen: 95.78516428571429
Score date d'arrivè moyen: 99.74171
Score ville d'origine moyen: 98.89881375
Score date de départ d'origine moyen: 89.86147000000001
